<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=9a8e9cb7026bcd1af4c745dd77c640baa1a8c9580ea61d3681cb22d3bb796256
  Stored in directory: /home/runner/.cache/pip/wheels/e3/3a/ee/4955a26c90a4b7deb6d725dc8ec7b8604a7aef44e43a2e8af7


Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.33.0
    Uninstalling protobuf-6.33.0:
      Successfully uninstalled protobuf-6.33.0


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.66
    Uninstalling yfinance-0.2.66:
      Successfully uninstalled yfinance-0.2.66
   ━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━ 1/2 [yfinance]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-10-29 09:13:00
-------------------
qualified stocks: 88
with latest results: 27
still star stocks: 17
-------------------
Initial Investment:  1.26 C
CY Investment:  1.53 C
Reserve:  1.48 L
Current:  1.45 C
-------------------
Today PnL: 617.00 (0.0%)
Current PnL: -19.35 L (-12.68%)
CY Booked + Current PnL: -7.22 L (-4.73%)
-------------------
Total profit:  2.45 L
Total loss:  -21.80 L
-------------------
Total Booked + Current PnL: 19.76 L (15.73%)
Total Booked PnL: 39.11 L (31.13%)
Curr Year Booked PnL: 12.13 L (8.37%)
Prev Year Booked PnL: 26.98 L (21.48%)
Est FTT:  2.32 C
Est FTT PnL: 87.05 L (60.09%)
Deployed:  1.26 C
Current:  1.45 C
CAGR/XIRR %: 8.48%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
79,TTKPRESTIG,770.0,117.72,76.0,M-SC,1.33,93816.0,-6961.0,7055.0,0.0,-6.91,7.52,0.09,245.0,-0.99,0.65,23.25,OX40N,NTT,DURABLES
59,RELIANCE,1533.0,-7.28,73.0,X-LC,4.18,231956.0,21590.0,7191.0,-0.0,10.26,3.10,13.68,37.0,3.00,1.62,28.48,XY25,NTT,REFINERIES
85,VOLTAS,1530.0,2.14,56.0,X-MC,3.94,214260.0,22518.0,15234.0,0.0,11.74,7.11,19.69,99.0,1.48,1.49,19.34,XY25,NTT,AC
2,ABBOTINDIA,35195.0,-9.64,41.0,X-MC,2.75,89310.0,-1368.0,16272.0,0.0,-1.51,18.22,16.44,101.0,-0.08,0.62,18.36,X40,ATH,PHARMA
42,ITC,452.0,-36.66,63.0,X-LC,1.42,204353.0,4215.0,16675.0,-0.0,2.11,8.16,10.44,4.0,0.25,1.43,7.79,X40,NTT,FMCG


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
75,TATAMOTORS,1065.00,-52.51,19.0,X-LC,18.69,149374.0,-127149.0,237221.0,0.35,-45.98,158.81,39.81,54.0,-0.54,1.04,5.28,XY24,NTT,AUTO
50,MASFIN,398.61,-20.51,45.0,H-SC,9.13,90510.0,-7470.0,29072.0,0.10,-7.62,32.12,22.05,152.0,-0.26,0.63,31.56,XR,ATH,FINANCE
0,5PAISA,593.00,93.51,30.0,H-SC,13.05,131706.0,-86790.0,114979.0,0.00,-39.72,87.30,12.90,153.0,-0.75,0.92,1.34,OX40N,NTT,FINANCE
1,ABB,7934.00,-40.55,47.0,H-LC,5.82,248424.0,-13195.0,132410.0,0.00,-5.04,53.30,45.57,7.0,-0.10,1.73,5.71,AR,NTT,ELECTRICAL
4,ALKYLAMINE,4546.37,-12.91,42.0,H-SC,5.54,81115.0,-19848.0,114380.0,-0.00,-19.66,141.01,93.63,148.0,-0.17,0.57,20.82,SR,ATH,CHEMICALS


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
0,5PAISA,593.00,93.51,30.0,H-SC,13.05,131706.0,-86790.0,114979.0,0.0,-39.72,87.30,12.90,153.0,-0.75,0.92,1.34,OX40N,NTT,FINANCE
1,ABB,7934.00,-40.55,47.0,H-LC,5.82,248424.0,-13195.0,132410.0,0.0,-5.04,53.30,45.57,7.0,-0.10,1.73,5.71,AR,NTT,ELECTRICAL
2,ABBOTINDIA,35195.00,-9.64,41.0,X-MC,2.75,89310.0,-1368.0,16272.0,0.0,-1.51,18.22,16.44,101.0,-0.08,0.62,18.36,X40,ATH,PHARMA
3,ACC,3906.00,-37.20,53.0,X-MC,2.56,186070.0,-51881.0,204528.0,0.0,-21.80,109.92,64.15,174.0,-0.25,1.30,4.38,XY24,BTT,CEMENT
4,ALKYLAMINE,4546.37,-12.91,42.0,H-SC,5.54,81115.0,-19848.0,114380.0,-0.0,-19.66,141.01,93.63,148.0,-0.17,0.57,20.82,SR,ATH,CHEMICALS


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
36,IEX,219.00,-29.93,63.0,H-SC,16.18,200185.0,231.0,96990.0,0.0,0.12,48.45,48.63,115.0,0.00,1.40,11.68,XR,NTT,MISC
57,RECLTD,446.00,45.36,48.0,M-LC,5.53,202922.0,-248.0,42370.0,-0.0,-0.12,20.88,20.74,55.0,-0.01,1.42,6.73,XY25,NTT,FINANCE
87,WIPRO,420.00,-14.70,46.0,M-LC,5.72,150276.0,-669.0,110122.0,0.0,-0.44,73.28,72.51,53.0,-0.01,1.05,5.55,XR,NTT,IT
32,HINDZINC,730.22,25.96,45.0,M-LC,11.23,204972.0,-104.0,112673.0,0.0,-0.05,54.97,54.89,52.0,-0.00,1.43,24.03,X5K,ATH,METALS


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
79,TTKPRESTIG,770.0,117.72,76.0,M-SC,1.33,93816.0,-6961.0,7055.0,0.0,-6.91,7.52,0.09,245.0,-0.99,0.65,23.25,OX40N,NTT,DURABLES
46,KANSAINER,340.0,-67.42,51.0,H-SC,2.70,224550.0,-45117.0,81444.0,0.0,-16.73,36.27,13.47,138.0,-0.55,1.57,14.18,XY24,NTT,PAINTS
74,TATAELXSI,9161.0,-22.20,55.0,H-MC,6.75,105355.0,-26480.0,68702.0,0.0,-20.09,65.21,32.03,98.0,-0.39,0.73,18.01,OX40N,NTT,IT
68,SIS,528.0,2060.62,50.0,H-SC,3.12,87197.0,-23835.0,47444.0,-0.0,-21.47,54.41,21.26,156.0,-0.50,0.61,17.81,OX40N,NTT,MISC
18,CERA,9475.0,-24.82,39.0,H-SC,2.66,137241.0,-38662.0,80684.0,0.0,-21.98,58.79,23.89,149.0,-0.48,0.96,18.53,OX40N,NTT,CERAMICS


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
12,BANKINDIA,190.0,-15.53,76.0,H-MC,16.24,216996.0,37188.0,75233.0,-0.0,20.68,34.67,62.52,88.0,0.49,1.51,61.82,XR,NTT,BANKS


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
62,SAIL,228.00,44.35,53.0,M-MC,9.78,227580.0,2618.0,165041.0,0.0,1.16,72.52,74.53,192.0,0.02,1.59,33.79,XY24,BTT,STEEL
47,KPIGREEN,731.05,10.56,72.0,H-SC,5.36,131708.0,6411.0,52512.0,0.0,5.12,39.87,47.03,141.0,0.12,0.92,64.91,MH,ATH,POWER
80,UNIONBANK,163.00,-6.98,66.0,M-LC,9.45,167219.0,26379.0,18110.0,0.0,18.73,10.83,31.59,66.0,1.46,1.17,50.18,XY24,NTT,BANKS
12,BANKINDIA,190.00,-15.53,76.0,H-MC,16.24,216996.0,37188.0,75233.0,-0.0,20.68,34.67,62.52,88.0,0.49,1.51,61.82,XR,NTT,BANKS
36,IEX,219.00,-29.93,63.0,H-SC,16.18,200185.0,231.0,96990.0,0.0,0.12,48.45,48.63,115.0,0.00,1.40,11.68,XR,NTT,MISC


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
12,BANKINDIA,190.00,-15.53,76.0,H-MC,16.24,216996.0,37188.0,75233.0,-0.0,20.68,34.67,62.52,88.0,0.49,1.51,61.82,XR,NTT,BANKS
86,WHIRLPOOL,2270.00,-38.77,63.0,M-SC,3.34,127070.0,12272.0,81770.0,-0.0,10.69,64.35,81.92,223.0,0.15,0.89,50.43,XR,NTT,DURABLES
47,KPIGREEN,731.05,10.56,72.0,H-SC,5.36,131708.0,6411.0,52512.0,0.0,5.12,39.87,47.03,141.0,0.12,0.92,64.91,MH,ATH,POWER
37,INDIAMART,4810.62,-52.96,59.0,H-SC,6.81,131461.0,8125.0,123508.0,-0.0,6.59,93.95,106.72,119.0,0.07,0.92,31.51,AR,ATH,MISC
32,HINDZINC,730.22,25.96,45.0,M-LC,11.23,204972.0,-104.0,112673.0,0.0,-0.05,54.97,54.89,52.0,-0.00,1.43,24.03,X5K,ATH,METALS


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
75,TATAMOTORS,1065.00,-52.51,19.0,X-LC,18.69,149374.0,-127149.0,237221.0,0.35,-45.98,158.81,39.81,54.0,-0.54,1.04,5.28,XY24,NTT,AUTO
13,BATAINDIA,2096.00,-38.87,32.0,X-SC,4.63,91383.0,-37287.0,82583.0,0.00,-28.98,90.37,35.20,219.0,-0.45,0.64,4.89,X40,NTT,FOOTWEAR
45,JSWINFRA,342.00,-20.69,36.0,X-MC,5.33,199091.0,-1256.0,31755.0,-0.00,-0.63,15.95,15.23,178.0,-0.04,1.39,31.42,X40N,NTT,REALTY
24,DMART,5391.45,-17.15,36.0,X-LC,9.91,101302.0,-1284.0,28091.0,0.00,-1.25,27.73,26.13,38.0,-0.05,0.71,24.00,X40N,ATH,FMCG
53,PGHH,17907.65,-30.55,37.0,X-MC,2.57,200970.0,150.0,67647.0,0.00,0.07,33.66,33.76,80.0,0.00,1.40,5.28,X40,ATH,FMCG


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
67,SIEMENS,4671.50,-3.25,44.0,H-LC,1.07,155485.0,-30610.0,78085.0,0.0,-16.45,50.22,25.51,15.0,-0.39,1.08,14.58,AR,ATH,ELECTRICAL
42,ITC,452.00,-36.66,63.0,X-LC,1.42,204353.0,4215.0,16675.0,-0.0,2.11,8.16,10.44,4.0,0.25,1.43,7.79,X40,NTT,FMCG
53,PGHH,17907.65,-30.55,37.0,X-MC,2.57,200970.0,150.0,67647.0,0.0,0.07,33.66,33.76,80.0,0.00,1.40,5.28,X40,ATH,FMCG
29,HAVELLS,2069.16,-6.89,42.0,X-MC,3.10,310926.0,-17065.0,123593.0,0.0,-5.20,39.75,32.48,92.0,-0.14,2.17,5.63,X40,ATH,ELECTRICAL
85,VOLTAS,1530.00,2.14,56.0,X-MC,3.94,214260.0,22518.0,15234.0,0.0,11.74,7.11,19.69,99.0,1.48,1.49,19.34,XY25,NTT,AC


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
10,BAJAJHFL,181.50,-15.11,42.0,X-MC,5.98,178776.0,-21730.0,118886.0,-0.0,-10.84,66.50,48.45,90.0,-0.18,1.25,3.09,X40N,ATH,FINANCE
21,COLPAL,3767.14,-4.32,45.0,X-MC,7.91,223270.0,-40095.0,153453.0,0.0,-15.22,68.73,43.04,84.0,-0.26,1.56,3.65,XY25,ATH,FMCG
52,PAGEIND,51769.93,-27.54,39.0,X-MC,6.82,81830.0,-1680.0,21709.0,0.0,-2.01,26.53,23.98,82.0,-0.08,0.57,3.69,X40,ATH,APPARELS
3,ACC,3906.00,-37.20,53.0,X-MC,2.56,186070.0,-51881.0,204528.0,0.0,-21.80,109.92,64.15,174.0,-0.25,1.30,4.38,XY24,BTT,CEMENT
84,VBL,671.64,-19.59,47.0,X-LC,9.31,289748.0,-26094.0,138760.0,0.0,-8.26,47.89,35.67,5.0,-0.19,2.02,4.45,X40N,ATH,FMCG


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
54,QUESS,986.00,-45.45,49.0,X-SC,38.23,53929.0,-11077.0,162008.0,0.0,-17.04,300.41,232.18,198.0,-0.07,0.38,5.19,XY24,NTT,MISC
52,PAGEIND,51769.93,-27.54,39.0,X-MC,6.82,81830.0,-1680.0,21709.0,0.0,-2.01,26.53,23.98,82.0,-0.08,0.57,3.69,X40,ATH,APPARELS
58,RELAXO,1176.00,-40.07,42.0,X-SC,4.85,81677.0,-63483.0,139414.0,0.0,-43.73,170.69,52.31,136.0,-0.46,0.57,9.33,X40N,NTT,FOOTWEAR
2,ABBOTINDIA,35195.00,-9.64,41.0,X-MC,2.75,89310.0,-1368.0,16272.0,0.0,-1.51,18.22,16.44,101.0,-0.08,0.62,18.36,X40,ATH,PHARMA
13,BATAINDIA,2096.00,-38.87,32.0,X-SC,4.63,91383.0,-37287.0,82583.0,0.0,-28.98,90.37,35.20,219.0,-0.45,0.64,4.89,X40,NTT,FOOTWEAR


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
76,TCS,4389.97,-27.72,55.0,X-LC,11.31,287443.0,-58513.0,125210.0,0.0,-16.91,43.56,19.28,1.0,-0.47,2.00,6.26,X40,ATH,IT
40,INFY,2275.00,-17.80,58.0,X-LC,6.18,319543.0,6655.0,165044.0,0.0,2.13,51.65,54.87,3.0,0.04,2.23,10.59,X40,BTT,IT
42,ITC,452.00,-36.66,63.0,X-LC,1.42,204353.0,4215.0,16675.0,-0.0,2.11,8.16,10.44,4.0,0.25,1.43,7.79,X40,NTT,FMCG
84,VBL,671.64,-19.59,47.0,X-LC,9.31,289748.0,-26094.0,138760.0,0.0,-8.26,47.89,35.67,5.0,-0.19,2.02,4.45,X40N,ATH,FMCG
1,ABB,7934.00,-40.55,47.0,H-LC,5.82,248424.0,-13195.0,132410.0,0.0,-5.04,53.30,45.57,7.0,-0.10,1.73,5.71,AR,NTT,ELECTRICAL


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
7,ASIANTILES,137.00,7127.78,64.0,L-SC,7.62,79317.0,-14493.0,91111.0,-0.0,-15.45,114.87,81.67,269.0,-0.16,0.55,53.60,XR,NTT,CERAMICS
20,COFFEEDAY,80.00,-45.67,50.0,L-SC,27.94,81773.0,-31776.0,71829.0,0.0,-27.98,87.84,35.27,268.0,-0.44,0.57,100.14,XR,NTT,HOTELS
50,MASFIN,398.61,-20.51,45.0,H-SC,9.13,90510.0,-7470.0,29072.0,0.1,-7.62,32.12,22.05,152.0,-0.26,0.63,31.56,XR,ATH,FINANCE
79,TTKPRESTIG,770.00,117.72,76.0,M-SC,1.33,93816.0,-6961.0,7055.0,0.0,-6.91,7.52,0.09,245.0,-0.99,0.65,23.25,OX40N,NTT,DURABLES
86,WHIRLPOOL,2270.00,-38.77,63.0,M-SC,3.34,127070.0,12272.0,81770.0,-0.0,10.69,64.35,81.92,223.0,0.15,0.89,50.43,XR,NTT,DURABLES


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
63,SAMMAANCAP,326.00,-197.73,79.0,M-SC,29.75,168534.0,19314.0,124867.0,0.0,12.94,74.09,96.62,208.0,0.15,1.18,82.39,XY25,NTT,FINANCE
69,SONACOMS,806.63,-31.93,72.0,M-SC,7.97,84761.0,-16398.0,56400.0,-0.0,-16.21,66.54,39.54,202.0,-0.29,0.59,19.52,AR,ATH,AUTO
5,ANGELONE,3033.00,17.35,68.0,X-SC,5.41,208108.0,17102.0,40602.0,0.0,8.95,19.51,30.21,157.0,0.42,1.45,31.70,X40N,NTT,FINANCE
12,BANKINDIA,190.00,-15.53,76.0,H-MC,16.24,216996.0,37188.0,75233.0,-0.0,20.68,34.67,62.52,88.0,0.49,1.51,61.82,XR,NTT,BANKS
47,KPIGREEN,731.05,10.56,72.0,H-SC,5.36,131708.0,6411.0,52512.0,0.0,5.12,39.87,47.03,141.0,0.12,0.92,64.91,MH,ATH,POWER


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,19.52
1,25,43.88
2,50,74.86


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    43.28
LC    32.19
MC    24.56
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     30.14
X40      16.26
XY25     12.09
XR       11.43
X40N     10.56
AR        7.80
OX40N     6.56
X200      1.76
X5K       1.43
SR        1.08
MH        0.92
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    24.52
X-LC    21.54
X-MC    18.77
M-SC    12.72
M-LC     5.07
X-SC     4.59
H-LC     4.57
H-MC     3.88
M-MC     1.59
L-SC     1.45
L-LC     1.01
L-MC     0.32
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,14.42,-4.25,38.27
IT,12.49,-17.41,80.39
FINANCE,12.14,-8.73,58.01
MISC,6.91,-16.15,80.51
ELECTRICAL,5.75,-9.45,48.25
BANKS,5.73,-4.08,65.57
PAINTS,5.52,-16.78,34.18
INSURANCE,3.87,0.21,37.00
AUTO,2.78,-44.07,104.20


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3255141.0,22
XR,1308871.0,14
AR,1170972.0,9
X40,825811.0,12
X40N,646390.0,9
OX40N,543640.0,9
XY25,449854.0,8
SR,266800.0,2
X5K,112673.0,1


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3319087.0,24
M-SC,1222090.0,15
X-MC,1187079.0,14
X-LC,1050681.0,13
X-SC,523220.0,6
H-MC,322343.0,3
M-LC,283275.0,4
H-LC,282767.0,3
L-SC,252905.0,3


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1146329.0      6
           AR         836686.0      5
           XR         773653.0      7
M-SC       XY24       701657.0      6
X-MC       XY24       578383.0      4
X-LC       X40        453860.0      5
           XY24       305205.0      2
X-MC       X40        289368.0      6
X-SC       X40N       278629.0      4
H-SC       SR         266800.0      2
           OX40N      243107.0      3
X-LC       X40N       217120.0      3
H-LC       AR         210495.0      2
H-MC       XY24       178408.0      1
X-MC       XY25       168687.0      2
M-MC       XY24       165041.0      1
L-SC       XR         162940.0      2
X-SC       XY24       162008.0      1
X-MC       X40N       150641.0      2
M-SC       OX40N      141866.0      4
           XR         129909.0      2
           XY25       124867.0      1
           AR         123791.0      2
M-LC       X5K        112673.0      1
           XR         110122.0      1
L-SC       OX40N       89965.0      1
X-SC       X40         82583.0      1
H-MC       XR          75233.0      1
X-LC       XY25        74496.0      3
H-LC       X200        72272.0      1
H-MC       OX40N       68702.0      1
L-MC       XR          57014.0      1
H-SC       MH          52512.0      1
M-LC       XY25        42370.0      1
L-LC       XY25        39434.0      1
M-LC       XY24        18110.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 56.0 seconds
